# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Data Statistics for OpenRewriteEval

In [ ]:
import zipfile
import pandas as pd
import editdistance
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
openrewriteeval_zippath = "/content/drive/My Drive/openrewriteeval_aaai24.zip" # @param {type:"string"}
with zipfile.ZipFile(openrewriteeval_zippath, 'r') as zip_ref:
    zip_ref.extractall('tmp')

df = pd.read_csv('tmp/openrewriteeval_aaai24.csv')

In [ ]:
def calculate(row):
  comment, source, target = row["comment"], row["source"], row["target"]
  editdist = editdistance.distance(source.split(" "), target.split(" "))
  metrics = {
      "editdistance": editdist,
      "editdistance_over_source": editdist/len(source.split(" ")),
      "editdistance_over_target": editdist/len(target.split(" ")),
      "editdistance_ratio": editdist/(len(source.split(" ")) + len(target.split(" "))),
      "word_len_comment": len(comment.split(" ")),
      "word_len_source": len(source.split(" ")),
      "word_len_target": len(target.split(" ")),
      "len_ratio": len(target)/len(source),
  }
  return pd.Series(metrics)

In [ ]:
new_fields_df = df.apply(calculate, axis=1)
result_df = pd.concat([df, new_fields_df], axis=1)

In [ ]:
fields = ["word_len_comment", "word_len_source", "word_len_target", "len_ratio", "editdistance", "editdistance_over_source"]
result_df_by_task = result_df.groupby("task")[fields].mean()
result_df_by_task